In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
import json
import time
from openai import OpenAI
client = OpenAI()

In [24]:
def choice_word():
    client_topic = OpenAI()

    # 게임이 시작되기 전에 주제 선정을 위한 chat api 호출
    topic_response = client_topic.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": """너는 단어를 골라주는 챗봇이야.
                                    *** 지시 사항 ***
                                    - 단어장 형식으로 만들 것
                                    - 단어 2개만 뽑을 것
                                    - 중복되는 단어는 안됨
                                    - 사물, 사람 이름, 가리지 않고 다양하게 섞어서 만들것
                                    

                                    *** 출력 형식 ***
                                    - 단어장 : [단어1, 단어2]

                                    *** 예시 ***
                                    - 단어장 : ['율곡 이이', '로제']

                        """
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "단어를 선정해서 json형식으로 넘겨줘 "
                    }
                ]
            }
        ],
        response_format={
            "type": "json_object"
        },
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    response = json.loads(topic_response.choices[0].message.content)
    return response['단어장'][0], response['단어장'][1]

In [25]:
# 게임 진행 함수
def play_game(word_for_user, word_for_bot):
    print(f'유저가 맞춰야할 단어 : {word_for_bot}')
    print(f'AI가 맞춰야할 단어 : {word_for_user}')
    # 프롬프트 작성
    system_prompt = f"""
    너는 단어 맞추기 게임을 하는 AI야. 
    사용자가 질문을 하면 헷갈리게 답변해줘.
    사용자가 맞춰야 할 단어는 '{word_for_bot}' 이거야

    <조건>
    - 장난가득한 말투로 대답할 것
    - 사실만을 말할 것
    - 사용자가 맞춰야 할 단어를 물어보지 말 것
    - 사용자가 단어를 유추하기 어렵게 답변해줄 것
    - 대답을 피하지 말 것
    """
    
    # 사용자의 질문 + 질문에 대한 ai 답변 리스트 변수
    user_to_ai_que_history = []
    ai_to_user_ans_history = []
    # ai의 질문 + 질문에 대한 사용자 답변 리스트 변수
    ai_to_user_que_history = []
    user_to_ai_ans_history = []
    # chat_list = []
    # user_que_history = []
    
    count = 0
    while True:
        count += 1
        print('='*50)
        print(f"게임 진행 횟수: {count}번째 턴")
        print('='*50)
        print('* 질문해주세요 *')
        # 사용자 질문
        time.sleep(3)
        user_question = input("사용자 질문: ")
        user_to_ai_que_history.append(user_question)
        user_prompt = f"""
        내가 맞춰야 될 단어는 '{user_question}'
        """
        print(f"사용자가 한 질문: {user_question}")
        # AI 답변 생성
        ai_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content":[{
                    "type": "text",
                    "text": system_prompt}]},
                {"role": "user", "content":[{
                    "type": "text",
                    "text": user_prompt
                }]}
            ],
            response_format={
                "type": "text"
            },
            temperature=0.5,
            max_tokens=2048,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        time.sleep(3)
        ai_answer = ai_response.choices[0].message.content
        print(f"AI 답변: {ai_answer}")
        ai_to_user_ans_history.append(ai_answer)
        
        # AI 질문 생성
        # chat_list.append(f"사용자 질문: {user_question}")
        # chat_list.append(f"AI 답변: {ai_answer}")
        
        system_prompt2 = f"""
        너는 단어 맞추기 게임을 하는 AI야. 
        이번엔 너가 맞춰야 할 단어에 대해 나한테 질문하는 차례야
        
        너가 맞춰야할 단어에 대한 지금까지 한 질문 내역 
        '{ai_to_user_que_history}',
        너가 맞춰야할 단어에 대한 지금까지 들은 답변 내역
        '{user_to_ai_ans_history}'

        <조건>
        - 정답과 관련된 질문만 할 것
        - 앞에서 했던 질문은 하지 말 것
        """
        
        ai_question_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": [{
                    "type": "text",
                    "text": system_prompt2}]},
                {"role": "user", "content": [{
                    "type": "text",
                    "text": '이번엔 너가 맞춰야 할 단어에 대해 나한테 질문해줘.'
                }]}
            ],
            response_format={
                "type": "text"
            },
            temperature=0.5,
            max_tokens=2048,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        time.sleep(3)
        ai_question = ai_question_response.choices[0].message.content
        print(f"AI가 한 질문: {ai_question}")
        ai_to_user_que_history.append(ai_question)
        # 질문에 대한사용자 답변
        user_answer = input("사용자의 질문에 대한 답변: ")
        user_to_ai_ans_history.append(user_answer)
        print(f"사용자가 한 답변: {user_answer}")

        print('='*50)
        print('******** 정답 맞추는 시간 ********')
        print('='*50)
        time.sleep(1)
        user_answer = input("사용자 정답 맞추기: ")
        if user_answer == word_for_bot:
            print("사용자가 AI의 단어를 맞췄습니다! 사용자가 승리했습니다!")
            print("""
□■□□□□□□□■□□□□□□□□□□□□■□□■□□□■■■■■■■■■□□□□□■□□□□□■□□□□□□□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□■■■■■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□■■■■■■■■□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□□□□□■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□□□□■■□□□□■□□□□■□□□□□□□■□□□□□□□■■■■■■■■■■□
□■□□□■■■■■□□□□□□□□□■■□■□□■□□□■■■■■■■■■□□□■■■■■□□□■□□□□■■□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□■■□
□■□□□■■■■■□□□□□□□□□■■□■■■■□□□□□□□■□□□□□□■■■□■■□□□■■■□□■■□□□□■■■□□■□□□□□□□■■■□□□□□□□□□□□□□■■□
□■□□□□□□□■□□□□□□□□□■■□■□□■□□□□□□□■□□□□□□■■□□□■■□□■■■□□■■■□□□■■■□□■□□□□□□□■□□□□□□□□□□□□□□□■■□
□■□□□□□□□■□□□□□□□□■■□□■□□■□■■■■■■■■■■■■□■■□□□■■□□■□□□■■■■■□□■□□□□■□□□□□□□■□□□□□□■■■■■■■■■■■□
□■□□□■■■□■□□□□□□□■■■□□■□□■□□□□□□□■□□□□□□■■□□□■■□□■□□■■□□■■■□■□□□□■□□□■■■□■□□□□□□□□□□□□□□□■■□
□■■■■■■■□■□□□□□□■■□□□□■□□■□□□□□□□■□□□□□□□■■■■■□□□■□■■□□□□■■□■□□□□■■■■■■■□■□□□□□□□□□□□□□□□■■□
□□□□□□□□□■□□□□□□■□□□□□■□□■□□□□□□■■■□□□□□□□□■□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□■■□
□□□□□□□□□■□□□□□□□□□□□□■□□■□□□□■■■□■■■□□□□□□□□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□
□□□□□□□□□■□□□□□□□□□□□□■□□■□□□■■■□□□■■■□□□□□□□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□
      """)
            break
        else: 
            print(f'사용자가 맞춰야할 답은 {user_answer}가 아닙니다. 다시 시도해주세요.')
       
        
        system_prompt3 = f"""
        너는 단어 맞추기 게임을 하는 AI야. 
        이번엔 질문과 답변을 바탕으로 단어를 맞출 차례야
        
        ---
        너가 맞춰야할 단어에 대한 지금까지 한 질문 내역:
        '{ai_to_user_que_history}'

        ---
        너가 맞춰야할 단어에 대한 지금까지 들은 답변 내역:
        '{user_to_ai_ans_history}'

        ---
        <조건>
        - 정답은 단어만 출력할 것
        """
        ai_ans_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": [{
                    "type": "text",
                    "text": system_prompt3}]},
                {"role": "user", "content": [{
                    "type": "text",
                    "text": '너가 지금까지 질문한 내역을 바탕으로 단어를 맞춰봐. 대답은 반드시 단어만 출력돼야해'
                }]}
            ],
            response_format={
                "type": "text"
            },
            temperature=0.5,
            max_tokens=2048,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        time.sleep(3)
        ans_word = ai_ans_response.choices[0].message.content
        print(f"AI의 답변: {ans_word}")
        
        # 단어 맞추기 확인
        if ans_word == word_for_user:
            print("AI가 단어를 맞췄습니다! AI가 승리했습니다!")
            print("""
□■□□□□□□□■□□□□□□□□□□□□■□□■□□□■■■■■■■■■□□□□□■□□□□□■□□□□□□□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□■■■■■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□■■■■■■■■□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□□□□□■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□□□□■■□□□□■□□□□■□□□□□□□■□□□□□□□■■■■■■■■■■□
□■□□□■■■■■□□□□□□□□□■■□■□□■□□□■■■■■■■■■□□□■■■■■□□□■□□□□■■□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□■■□
□■□□□■■■■■□□□□□□□□□■■□■■■■□□□□□□□■□□□□□□■■■□■■□□□■■■□□■■□□□□■■■□□■□□□□□□□■■■□□□□□□□□□□□□□■■□
□■□□□□□□□■□□□□□□□□□■■□■□□■□□□□□□□■□□□□□□■■□□□■■□□■■■□□■■■□□□■■■□□■□□□□□□□■□□□□□□□□□□□□□□□■■□
□■□□□□□□□■□□□□□□□□■■□□■□□■□■■■■■■■■■■■■□■■□□□■■□□■□□□■■■■■□□■□□□□■□□□□□□□■□□□□□□■■■■■■■■■■■□
□■□□□■■■□■□□□□□□□■■■□□■□□■□□□□□□□■□□□□□□■■□□□■■□□■□□■■□□■■■□■□□□□■□□□■■■□■□□□□□□□□□□□□□□□■■□
□■■■■■■■□■□□□□□□■■□□□□■□□■□□□□□□□■□□□□□□□■■■■■□□□■□■■□□□□■■□■□□□□■■■■■■■□■□□□□□□□□□□□□□□□■■□
□□□□□□□□□■□□□□□□■□□□□□■□□■□□□□□□■■■□□□□□□□□■□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□■■□
□□□□□□□□□■□□□□□□□□□□□□■□□■□□□□■■■□■■■□□□□□□□□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□
□□□□□□□□□■□□□□□□□□□□□□■□□■□□□■■■□□□■■■□□□□□□□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□
      """)
            break
        else:
            print(f'AI가 맞춰야할 답은 {ans_word}가 아닙니다. 다시 시도해주세요.')

In [26]:
# 단어 선택
word_for_user, word_for_bot = choice_word()

In [27]:
play_game(word_for_user, word_for_bot)

유저가 맞춰야할 단어 : 마리아
AI가 맞춰야할 단어 : 태양
게임 진행 횟수: 1번째 턴
* 질문해주세요 *
사용자가 한 질문: 사람이야?
AI 답변: 오호, 사람이라니! 그건 정말 다양한 모습으로 나타날 수 있는 존재지. 하지만 내가 말하고 싶은 건, 그건 특정한 이름을 가진 존재야. 아마도 한 사람의 이름이기도 하고, 또 어떤 노래의 제목이기도 해! 흥미롭지? 계속 생각해봐!
AI가 한 질문: 이 단어는 사람이나 동물과 관련이 있나요?
사용자가 한 답변: 
******** 정답 맞추는 시간 ********
사용자가 맞춰야할 답은 몰라요잉가 아닙니다. 다시 시도해주세요.
AI의 답변: 없음
AI가 맞춰야할 답은 없음가 아닙니다. 다시 시도해주세요.
게임 진행 횟수: 2번째 턴
* 질문해주세요 *
사용자가 한 질문: 여자야?
AI 답변: 오호! 여자라니, 그건 참 재미있는 질문이네! 하지만 내가 알고 있는 그 단어는, 어떤 사람의 이름이기도 해. 그리고 그 이름은 여러 나라에서 많이 쓰이기도 해. 아마 너도 한 번쯤은 들어봤을 법한 이름일걸? 힌트는 그 이름이 '마'로 시작된다는 거야! 흥미진진하네!
AI가 한 질문: 이 단어는 자연에서 발견되나요?
사용자가 한 답변: 어 우리가 살고 있는 은하계에서 유일한 항성이야 제발 맞쳐줘
******** 정답 맞추는 시간 ********
사용자가 맞춰야할 답은 또 몰아요잉가 아닙니다. 다시 시도해주세요.
AI의 답변: 태양
AI가 단어를 맞췄습니다! AI가 승리했습니다!

□■□□□□□□□■□□□□□□□□□□□□■□□■□□□■■■■■■■■■□□□□□■□□□□□■□□□□□□□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□■■■■■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□■■■■■■■■□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□□□□□■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□□□□■■□□□□■□□□□■□

In [28]:
!pip install art pywhatkit

In [29]:
from art import text2art, tprint
print(text2art("you lose"))

                       _                   
 _   _   ___   _   _  | |  ___   ___   ___ 
| | | | / _ \ | | | | | | / _ \ / __| / _ \
| |_| || (_) || |_| | | || (_) |\__ \|  __/
 \__, | \___/  \__,_| |_| \___/ |___/ \___|
 |___/                                     



In [30]:
import pywhatkit as kit
kit.image_to_ascii_art("./image/image.png", "you lose.txt")

'######&&&&&&&&&&&&&&&&&@&@@@@@&&@@&&&@@@@@@@@&&@&&&&&&@@@@@@$$@@@$$%$$%%*$*!%*!!\n###############################@$&&&#&&&&&&&&&&&&&&&&&&&&@@@@$@@@@@@$!:!!*!!!!!!\n&@@&&&&&&@&&&&&&&&&&&&&&&&&&&&@$@@@@&#########SSS#########&&&&&&&@@@@***%$*!**%$\n&&@@##&&&@@##&&&@&#&&&&@&&&@@@@&&@@@@@@$$$@@@&@@&&######&&&&&&&&&&&&&&&&#&&@@@$*\n&&&@@&&&&@@&&&&&@@#&&&&$&#@$$@@@@@@@&&&*!!%*$$%*%&######&$@&#########&########&@\n&&&&@@&&&&@@&&&&&$&#&&&@&@%%%$%%%%$$$$@@!*%%$*!*$&######&&##########&&&&#####&##\n@&&&&@@&&&&@$&&&&@@&&&&@$%**$$%%%%%%%%%$%%*$*::!%@############################&#\n&@@&&&&@&&&&@@##&&@@#&&&$***$%*%$$%%%%%$$%%$$%!*%@######SS######################\n&&@@&&&&@&&&&@@&&&&@&&&&@$*****%$%%$$$%$%%$#S#*%@&#####SS####&##&&&&############\n&&&&@&&&&@&&&&@&&&&&&&&&&@$****%$%%%%%*%$$%$@&%%%@######@$$$@@&&&###############\n&&&&&&&&&&&&&&&@&&&&&####@&%**%%%$$%%%$$$%!!*@%!*@#####@%**%@@&&&&&&#####S##S###\n&$%$&@$$@&@@@&&&&####SSS##@@%%%%@&@%%$&$$**!%%*%%$####&$****$$$@&&&&########SSSS\n&@@@&@$@@&@@@##

In [31]:
print("""
□■□□□□□□□■□□□□□□□□□□□□■□□■□□□■■■■■■■■■□□□□□■□□□□□■□□□□□□□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□■■■■■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□■■■■■■■■□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□□□□□■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□□□□■■□□□□■□□□□■□□□□□□□■□□□□□□□■■■■■■■■■■□
□■□□□■■■■■□□□□□□□□□■■□■□□■□□□■■■■■■■■■□□□■■■■■□□□■□□□□■■□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□■■□
□■□□□■■■■■□□□□□□□□□■■□■■■■□□□□□□□■□□□□□□■■■□■■□□□■■■□□■■□□□□■■■□□■□□□□□□□■■■□□□□□□□□□□□□□■■□
□■□□□□□□□■□□□□□□□□□■■□■□□■□□□□□□□■□□□□□□■■□□□■■□□■■■□□■■■□□□■■■□□■□□□□□□□■□□□□□□□□□□□□□□□■■□
□■□□□□□□□■□□□□□□□□■■□□■□□■□■■■■■■■■■■■■□■■□□□■■□□■□□□■■■■■□□■□□□□■□□□□□□□■□□□□□□■■■■■■■■■■■□
□■□□□■■■□■□□□□□□□■■■□□■□□■□□□□□□□■□□□□□□■■□□□■■□□■□□■■□□■■■□■□□□□■□□□■■■□■□□□□□□□□□□□□□□□■■□
□■■■■■■■□■□□□□□□■■□□□□■□□■□□□□□□□■□□□□□□□■■■■■□□□■□■■□□□□■■□■□□□□■■■■■■■□■□□□□□□□□□□□□□□□■■□
□□□□□□□□□■□□□□□□■□□□□□■□□■□□□□□□■■■□□□□□□□□■□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□■■□
□□□□□□□□□■□□□□□□□□□□□□■□□■□□□□■■■□■■■□□□□□□□□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□
□□□□□□□□□■□□□□□□□□□□□□■□□■□□□■■■□□□■■■□□□□□□□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□□□□
      """)


□■□□□□□□□■□□□□□□□□□□□□■□□■□□□■■■■■■■■■□□□□□■□□□□□■□□□□□□□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□■■■■■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□■■■■■■■■□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□□□□
□■□□□□□□□■□□□□□□□□□□■□■□□■□□□■■□□□□□■■□□■■■■■■■□□■□□□□■■□□□□■□□□□■□□□□□□□■□□□□□□□■■■■■■■■■■□
□■□□□■■■■■□□□□□□□□□■■□■□□■□□□■■■■■■■■■□□□■■■■■□□□■□□□□■■□□□□■□□□□■□□□□□□□■□□□□□□□□□□□□□□□■■□
□■□□□■■■■■□□□□□□□□□■■□■■■■□□□□□□□■□□□□□□■■■□■■□□□■■■□□■■□□□□■■■□□■□□□□□□□■■■□□□□□□□□□□□□□■■□
□■□□□□□□□■□□□□□□□□□■■□■□□■□□□□□□□■□□□□□□■■□□□■■□□■■■□□■■■□□□■■■□□■□□□□□□□■□□□□□□□□□□□□□□□■■□
□■□□□□□□□■□□□□□□□□■■□□■□□■□■■■■■■■■■■■■□■■□□□■■□□■□□□■■■■■□□■□□□□■□□□□□□□■□□□□□□■■■■■■■■■■■□
□■□□□■■■□■□□□□□□□■■■□□■□□■□□□□□□□■□□□□□□■■□□□■■□□■□□■■□□■■■□■□□□□■□□□■■■□■□□□□□□□□□□□□□□□■■□
□■■■■■■■□■□□□□□□■■□□□□■□□■□□□□□□□■□□□□□□□■■■■■□□□■□■■□□□□■■□■□□□□■■■■■■■□■□□□□□□□□□□□□□□□■■□
□□□□□□□□□■□□□□□□■□□□□□■□□■□□□□□□■■■□□□□□□□□■□□□□□■□□□□□□□□□□■□□□□□□□□□□□□■□□□□□□□□□□□□□□□■■□
□□□□□□□□□■□□□□□□□□□□□□■□□■□□□□■■■□■■■□□□□□□□□□□□□■□□□□□□□□□□■□□□□□□□□

In [32]:
!pip install art

In [33]:
from art import text2art

text = "너 개못하자나"
ascii_art = text2art(text, font="block")  # 다양한 폰트 사용 가능
print(ascii_art)